In [14]:
#!/bin/python
import argparse
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np
import sys
import sklearn.model_selection
from sklearn.preprocessing import LabelEncoder
import pickle
import xgboost as xgb
import tensorflow as tf
import numpy as np
from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.metrics import confusion_matrix, mean_squared_error, precision_score, accuracy_score


from xgboost import XGBClassifier

# argument parsing 
#parser = argparse.ArgumentParser(description='Process some integers.')
#parser.add_argument('integers', metavar='N', type=int, nargs='+',
#                   help='an integer for the accumulator')
#parser.add_argument('--sum', dest='accumulate', action='store_const',
#                   const=sum, default=max,
#                   help='sum the integers (default: find the max)')

#args = parser.parse_args()
#idx_lr = sys.argv[1]
idx_lr = 1
#idx_md = sys.argv[2]
idx_md = 1

# set seed 
rng = np.random.RandomState(1)

# set directory
os.getcwd()
os.chdir("/ysm-gpfs/pi/zhao/zy92/projects/ddipred/ddi_pred/data")

# list of desired DDI Types
desired_DDI = [0, 1, 2, 3, 4, 5, 6, 7, 15, 16, 17, 18, 19, 20, 21, 22, 26, 28, 30, 31, 32, 38, 40, 41, 43, 44, 45,
               49, 50, 51, 52, 54, 55, 62, 67, 68, 72, 74, 76, 78, 79, 80, 81]


# parameters
parameters = {"learning_rate"    : [0.01, 0.10, 0.25, 0.5, 1 ],
    "max_depth"        : [ 3, 4, 5, 6, 7],
    "min_child_weight" : [ 1, 3, 5, 7 ],
    "gamma"            : [ 0.0, 0.1, 0.2 , 0.3, 0.4 ],
    "colsample_bytree" : [ 0.3, 0.4, 0.5 , 0.7 ] }

# load the data 
ddidata = pd.read_excel("DrugBank_known_ddi.xlsx")
interactiondict = pd.read_csv("Interaction_information.csv")
safe_drugs = pd.read_csv("safe_drug_combos.csv")
drug_similarity_feature = pd.read_csv("drug_similarity.csv")
drug_similarity = drug_similarity_feature.iloc[:, 1:len(drug_similarity_feature)+1]

# filter ddidata for desired DDI types
up_ddidata = ddidata[ddidata.Label.isin(desired_DDI)]
new_ddidata = up_ddidata.copy()

# convert types to int
new_ddidata.drug1 = up_ddidata.drug1.str[2:].astype(int)
new_ddidata.drug2 = up_ddidata.drug2.str[2:].astype(int)
new_ddidata.Label = up_ddidata.Label


# incorporate safe_drugs into new_ddidata with DDIType 0
safe_drugs["Label"] = 0

frames = [safe_drugs, new_ddidata]
ddi_df = pd.concat(frames)

# create a DB to index dictionary from similarity dataset
DB_to_index = {}
i = 0
for col in drug_similarity.columns:
    DB_to_index[int(col[2:7])] = i
    i = i + 1

# filter output to only include DBs with similarity features
ddi_df_output = ddi_df[ddi_df.drug1.isin(DB_to_index)]
ddi_output = ddi_df_output[ddi_df_output.drug2.isin(DB_to_index)]

# filter out the duplicate samples
bool_series_to_delete = ddi_output[['drug1', 'drug2']].duplicated()
ddi_clean = ddi_output[~bool_series_to_delete]


# feature building 
# add similarity feature for each drug-drug pair
n_similarity = 2159
sim_array = np.empty([ddi_clean.shape[0], 2*n_similarity], dtype='float16')
i = 0
for index, (_, row) in enumerate(ddi_clean.iterrows()):
    if index % 10000 == 0:
        print("INFO: iter " + str(index + 1))
    drug1_index = DB_to_index[row["drug1"]]
    drug2_index = DB_to_index[row["drug2"]]
    feature_vec = np.hstack([drug_similarity.iloc[:,drug1_index],drug_similarity.iloc[:,drug2_index]])
    sim_array[index, ] = feature_vec
    
# create input and output vectors for training

X_data = sim_array
y_data = np.array(ddi_clean.Label)

# transform the y
encoder = LabelEncoder()
encoder.fit(y_data)
encoded_Y = encoder.transform(y_data)

#y_data = tf.keras.utils.to_categorical(encoded_Y)
y_data = encoded_Y

X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X_data, y_data, test_size = 0.3
                                                                           , random_state = 1)
X_test, X_val, y_test, y_val = sklearn.model_selection.train_test_split(X_test, y_test, test_size = 0.5
                                                                       , random_state = 1)

# dmatrix
dtrain = xgb.DMatrix(X_train, label = y_train)
dtest = xgb.DMatrix(X_test, label = y_test)
dval = xgb.DMatrix(X_val, label = y_val)

print("----- INFO: training data preprocess is done. -----")
# param
max_depth = parameters["max_depth"][idx_md]
learning_rate = parameters["learning_rate"][idx_lr]


param = {
    'max_depth': max_depth,  # the maximum depth of each tree
    'learning_rate': learning_rate,
    'eta': 0.3,  # the training step for each iteration
    #'silent': 3,  # logging mode - quiet
    'verbosity': 3,  # debug mode
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'eval_metric': ['mae', 'mlogloss'],
    'num_class': 43}  # the number of classes that exist in this datset
num_round = 1  # the number of training iterations

print("----- INFO: XGBoost Parameters -----")
print(param)
# training
history_dict = {}
bst = xgb.train(param, 
                dtrain, 
                num_round,
                early_stopping_rounds = 5,
                evals = [(dtest,'test'), (dtrain,'train')],
                evals_result = history_dict
                )



# testing
preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])
preds = bst.predict(dtest)
best_preds = np.asarray([np.argmax(line) for line in preds])

# save the object 
print("----- INFO: saving results -----")
output_dir = "/ysm-gpfs/pi/zhao/zy92/projects/ddipred/ddi_pred/code/xgboost/output/"
history_path = output_dir + "_lr_" + learning_rate + "_md_" + max_depth + "_history_dict.pickle"
model_path = output_dir + "_lr_" + learning_rate + "_md_" + max_depth + "_xgb.pickle"
pickle.dump(history_dict, history_path)
pickle.dump(bst, model_path)




[18:34:51] ======== Monitor: Learner ========
[18:34:51] ======== Monitor: GBTree ========
[18:34:51] ======== Monitor: Learner ========
[18:34:51] Configure: 0.001375s, 1 calls @ 1375us

[18:34:51] WARNING: /workspace/src/common/timer.cc:88: Timer for EvalOneIter did not get stopped properly.
[18:34:51] GetGradient: 0.060069s, 1 calls @ 60069us

[18:34:51] PredictRaw: 18.0722s, 1 calls @ 18072213us

[18:34:51] UpdateOneIter: 276.569s, 1 calls @ 276569098us

[18:34:51] ======== Monitor: GBTree ========
[18:34:51] BoostNewTrees: 234.052s, 1 calls @ 234051795us

[18:34:51] CommitModel: 24.3829s, 1 calls @ 24382897us

[18:34:51] ======== Monitor: Learner ========
[18:34:51] Configure: 0.001229s, 1 calls @ 1229us

[18:34:51] WARNING: /workspace/src/common/timer.cc:88: Timer for EvalOneIter did not get stopped properly.
[18:34:51] GetGradient: 0.060403s, 1 calls @ 60403us

[18:34:51] PredictRaw: 16.6614s, 1 calls @ 16661366us

[18:34:51] UpdateOneIter: 304.127s, 1 calls @ 304126857us

[18:3

XGBoostError: [18:41:59] /workspace/src/metric/elementwise_metric.cu:332: Check failed: preds.Size() == info.labels_.Size() (657857 vs. 15299) : label and prediction size not match, hint: use merror or mlogloss for multi-class classification
Stack trace:
  [bt] (0) /home/zy92/anaconda3/lib/python3.7/site-packages/xgboost/./lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x54) [0x2ae34dd42614]
  [bt] (1) /home/zy92/anaconda3/lib/python3.7/site-packages/xgboost/./lib/libxgboost.so(xgboost::metric::EvalEWiseBase<xgboost::metric::EvalRowMAE>::Eval(xgboost::HostDeviceVector<float> const&, xgboost::MetaInfo const&, bool)+0x15a) [0x2ae34dfa8c6a]
  [bt] (2) /home/zy92/anaconda3/lib/python3.7/site-packages/xgboost/./lib/libxgboost.so(xgboost::LearnerImpl::EvalOneIter(int, std::vector<xgboost::DMatrix*, std::allocator<xgboost::DMatrix*> > const&, std::vector<std::string, std::allocator<std::string> > const&)+0x419) [0x2ae34de2efb9]
  [bt] (3) /home/zy92/anaconda3/lib/python3.7/site-p

In [19]:
param = {
    'max_depth': max_depth,  # the maximum depth of each tree
    'learning_rate': learning_rate,
    'eta': 0.3,  # the training step for each iteration
    #'silent': 3,  # logging mode - quiet
    'verbosity': 3,  # debug mode
    'objective': 'multi:softprob',  # error evaluation for multiclass training
    'eval_metric': ['merror', 'mlogloss'],
    'num_class': 43}  # the number of classes that exist in this datset
num_round = 1  # the number of training iterations

history_dict = {}
bst = xgb.train(param, 
                dtrain, 
                num_round,
                early_stopping_rounds = 5,
                evals = [(dval, 'validation'), (dtest,'test'), (dtrain,'train')],
                #evals = [(dtest,'test'), (dtrain,'train')],
                evals_result = history_dict
                )



# testing
#preds = bst.predict(dtest)
#best_preds = np.asarray([np.argmax(line) for line in preds])
#preds = bst.predict(dtest)
#best_preds = np.asarray([np.argmax(line) for line in preds])

# save the object 
print("----- INFO: saving results -----")
output_dir = "/ysm-gpfs/pi/zhao/zy92/projects/ddipred/ddi_pred/code/xgboost/output/"
history_path = output_dir + "_lr_" + str(learning_rate) + "_md_" + str(max_depth) + "_history_dict.pickle"
model_path = output_dir + "_lr_" + str(learning_rate) + "_md_" + str(max_depth) + "_xgb.pickle"
history_fo = open(history_path, "wb")
model_fo = open(model_path, "wb")
pickle.dump(history_dict, history_fo)
pickle.dump(bst, model_fo)
history_fo.close()
model_fo.close()


[19:39:33] DEBUG: /workspace/src/gbm/gbtree.cc:146: Using tree method: 2
[19:39:39] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=4
[19:39:43] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 4 extra nodes, 0 pruned nodes, max_depth=2
[19:39:49] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=4
[19:39:56] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 12 extra nodes, 0 pruned nodes, max_depth=4
[19:40:02] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 16 extra nodes, 0 pruned nodes, max_depth=4
[19:40:08] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 20 extra nodes, 0 pruned nodes, max_depth=4
[19:40:14] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 10 extra nodes, 0 pruned nodes, max_depth=4
[19:40:20] INFO: /workspace/src/tree/updater_prune.cc:89: tree pruning end, 6 extra node

In [7]:
type(dtrain)

xgboost.core.DMatrix